In [1]:
use Modern::Perl;
use HackaMol;
use Math::Vector::Real;

# Benzene is more than 12 vectors: using HackaMol to work with atomic "metadata" and coordinates.

<img src="fatstacks.jpg">

Suggested Prerequisites:

1. This notebook builds directly from the [p5-Vectors-Benzene notebook](https://github.com/demianriccardi/EC-Chem452/blob/master/Notebooks/p5-Vectors-Benzene_2016_02_26.ipynb). 

2. This notebook uses the HackaMol library.  Please review the [Readme](https://github.com/demianriccardi/HackaMol) and the linked [HackaMol paper](https://www.researchgate.net/profile/Demian_Riccardi/publication/273778191_HackaMol_an_object-oriented_Modern_Perl_library_for_molecular_hacking_on_multiple_scales/links/550ebec60cf27526109e6ade.pdf) for additional information about the library.


### Ability to work with vectors while maintaining a connection to the atomic metadata is prerequisite to carrying out structural analysis of biological molecules.
At a given temperature, pressure, and molar constitution $G(T,P,n_1,n_2 ... )$, a biological molecule (or collection of molecules) has a space of available conformations or configurations that may be large ($S=k\ln W$).  If you were to carry out a molecular dynamics simulation (which we will come back to later), you would most likely prepare your system using a single structure provided by X-ray crystallography or NMR spectroscopy, solvate your system in water (maybe with salt ions), and then add some heat so that it wiggles about.  

When simulated in a computer, a condensed phase system is just a vector space that varies with time (molecular dynamics) or conformation (monte carlo) that is restrained (or even constrained) by the information of how the vectors are assumed to interact (e.g. this water hydrogen, with these coordinates, is bound to this water oxygen with these coordinates). While this may seem an obfuscated way to describe computational work with molecules, it is a valuable perspective.  The hierarchy of chemistry and biochemistry (atom, atoms, bonds, molecule, molecules, polymers, nonbonded interactions, etc.) provide the information that biases that which is possible in empirical computational work (i.e. classical mechanical molecular simulations). 

#### Lab exercise: Fun with benzene atoms and coordinates using vector analysis
I wrote the HackaMol library to make my computational life easier, more fun, more sophisticated, and more reproducible.  HackaMol provides a molecular object system that packages atomic and molecular data into classes that have methods for interacting with the data.  In the previous exercise, we stripped benzene down to nothing but a a collection of 12 vectors.  Now we will add atomic information back to these vectors.  We'll call this information "metadata"; metadata is data that describes other data.  We can add such information to those 12 vectors to name atoms, add bonds, angles, and dihedrals all with names and information. We will do that here.   

In this lab we will use benzene to interact with the hierarchy [HackaMol](https://metacpan.org/pod/HackaMol) classes: HackaMol::Atom (https://metacpan.org/pod/HackaMol::Atom), [HackaMol::AtomGroup](https://metacpan.org/pod/HackaMol::AtomGroup), [HackaMol::Bond](https://metacpan.org/pod/HackaMol::Bond), [HackaMol::Angle](https://metacpan.org/pod/HackaMol::Angle), [HackaMol::Dihedral](https://metacpan.org/pod/HackaMol::Dihedral), and [HackaMol::Molecule](https://metacpan.org/pod/HackaMol::Molecule). The HackaMol::Atom objects contain coordinates which are just Math::Vector::Real objects;  the other classes listed above contain groups of HackaMol::Atom objects plus other things.  

#### Mental exercise: 
Before going an further.  Again, take a look at the PDB file for a mutant of T4 Lysozyme [PDBID: 1L54](http://www.rcsb.org/pdb/files/1L54.pdb). Find the lines of coordinates that also contain additional metadata. What are all the different types of metadata that you can see in this file?


### Let's define a string (c6h6_xyz) that contains the xyz file description of a benzene molecule:

In [2]:
my $c6h6_xyz =
'  C        0.00000        1.40272        0.00000
  C       -1.21479        0.70136        0.00000
  C       -1.21479       -0.70136        0.00000
  C        0.00000       -1.40272        0.00000
  C        1.21479       -0.70136        0.00000
  C        1.21479        0.70136        0.00000
  H        0.00000        2.49029        0.00000
  H       -2.15666        1.24515        0.00000
  H       -2.15666       -1.24515        0.00000
  H        0.00000       -2.49029        0.00000
  H        2.15666       -1.24515        0.00000
  H        2.15666        1.24515        0.00000
'; 

print $c6h6_xyz;

  C        0.00000        1.40272        0.00000
  C       -1.21479        0.70136        0.00000
  C       -1.21479       -0.70136        0.00000
  C        0.00000       -1.40272        0.00000
  C        1.21479       -0.70136        0.00000
  C        1.21479        0.70136        0.00000
  H        0.00000        2.49029        0.00000
  H       -2.15666        1.24515        0.00000
  H       -2.15666       -1.24515        0.00000
  H        0.00000       -2.49029        0.00000
  H        2.15666       -1.24515        0.00000
  H        2.15666        1.24515        0.00000


1


The coordinates are the same as those of the last notebook, except now we're labeling them with element symbols.

### Load this xyz file string into two HackaMol::Molecule objects into the variables \$bnz1 and \$bnz2:

In [3]:
my $bnz1 = HackaMol->new->read_string_mol($c6h6_xyz, 'xyz');
$bnz1->name("Benzene");

Benzene


`HackaMol->new` provides a builder that has all sorts of methods, one being  `read_string_mol`.  We can store the builder in a variable to avoid creating it every time.  Let's make another two benzene molecules, but this time let's use a builder.  


In [4]:
my $bldr = HackaMol->new; # a builder object

my $bnz2 = $bldr->read_string_mol($c6h6_xyz, 'xyz');
my $bnz3 = $bldr->read_string_mol($c6h6_xyz, 'xyz');

$_->name("Benzene") foreach ($bnz2,$bnz3);  # name each of them benzene

say $bnz2->name;

Benzene


1


\$bnz1, \$bnz2, and \$bnz3 are all HackaMol::Molecule objects that have some data and methods for interacting with the data.  While both molecules have all the same values, they are independent of one another.  Currently we have two identical, unique copies of the data packaged in HackaMol::Molecule objects.   Let's dump \$bnz1 out to take a look at it's guts:  

In [5]:
print $bnz1->dump;

$VAR1 = bless( {
                 't' => 0,
                 'atoms' => [
                              bless( {
                                       'symbol' => 'C',
                                       'iatom' => 0,
                                       'name' => 'at0',
                                       'bond_count' => 0,
                                       't' => 0,
                                       'coords' => [
                                                     bless( [
                                                              '0',
                                                              '1.40272',
                                                              '0'
                                                            ], 'Math::Vector::Real' )
                                                   ]
                                     }, 'HackaMol::Atom' ),
                              bless( {
                                       'bond_count' => 0

1


 Looking at the dumped information, you can see that the HackaMol::Molecue has a collection of atoms, each of which is a HackaMol::Atom object, and each atom has coordinates, which are just Math::Vector::Real objects.  There are some other attributes that you will learn about as you progress.  
 
 `$bnz1->dump` calls the `dump` method of \$bnz1. Dump is available for all HackaMol classes. Let's use another method (`get_atoms`) to grab the first atom and then use the atom's dump method. 

In [6]:
print $bnz1->get_atoms(0)->dump;

$VAR1 = bless( {
                 'symbol' => 'C',
                 'iatom' => 0,
                 'name' => 'at0',
                 'bond_count' => 0,
                 't' => 0,
                 'coords' => [
                               bless( [
                                        '0',
                                        '1.40272',
                                        '0'
                                      ], 'Math::Vector::Real' )
                             ]
               }, 'HackaMol::Atom' );


1


You can adjust the index from 0 to 1, 2, 3 .. 11 to dump out the information for each atom, in turn. Let's pull out the coordinates (Math::Vector::Real) objects and reproduce a little code from the last notebook.

In [7]:
my @vecs1 = map {$_->xyz} $bnz1->all_atoms;

{0, 1.40272, 0}{-1.21479, 0.70136, 0}{-1.21479, -0.70136, 0}{0, -1.40272, 0}{1.21479, -0.70136, 0}{1.21479, 0.70136, 0}{0, 2.49029, 0}{-2.15666, 1.24515, 0}{-2.15666, -1.24515, 0}{0, -2.49029, 0}{2.15666, -1.24515, 0}{2.15666, 1.24515, 0}


In the previous cell, I used the `map` function, which may be my favorite function in Perl. Map takes a list, acts on it element by element and then passes the result to the left as a list.  
[`$bnz1->all_atoms`] sends all the atoms to the map which pulls the xyz coordinates and passes them to the left.  This map has transformed the list of atoms into a stripped down list of vectors.  All this is captured in the `@vecs` variable, which is an array.  This array is effectively the same array as that in the last notebook. Let's use two maps to translate the vectors of \$bnz2:

In [8]:
my @vecs2 = map {$_->xyz + V(0,0,10)} $bnz2->all_atoms;

{0, 1.40272, 10}{-1.21479, 0.70136, 10}{-1.21479, -0.70136, 10}{0, -1.40272, 10}{1.21479, -0.70136, 10}{1.21479, 0.70136, 10}{0, 2.49029, 10}{-2.15666, 1.24515, 10}{-2.15666, -1.24515, 10}{0, -2.49029, 10}{2.15666, -1.24515, 10}{2.15666, 1.24515, 10}


In [9]:
foreach my $iv (0 .. $#vecs1){
    foreach my $jv ($iv+1 .. $#vecs1){
        printf("from bnz1:  %5i%5i%10.3f\n",$iv, $jv, abs($vecs1[$iv]-$vecs1[$jv]));
        printf("from bnz2:  %5i%5i%10.3f\n",$iv, $jv, abs($vecs2[$iv]-$vecs2[$jv]));
        printf("from bnz12: %5i%5i%10.3f\n",$iv, $jv, abs($vecs1[$iv]-$vecs2[$jv]));
    }
}

from bnz1:      0    1     1.403
from bnz2:      0    1     1.403
from bnz12:     0    1    10.098
from bnz1:      0    2     2.430
from bnz2:      0    2     2.430
from bnz12:     0    2    10.291
from bnz1:      0    3     2.805
from bnz2:      0    3     2.805
from bnz12:     0    3    10.386
from bnz1:      0    4     2.430
from bnz2:      0    4     2.430
from bnz12:     0    4    10.291
from bnz1:      0    5     1.403
from bnz2:      0    5     1.403
from bnz12:     0    5    10.098
from bnz1:      0    6     1.088
from bnz2:      0    6     1.088
from bnz12:     0    6    10.059
from bnz1:      0    7     2.162
from bnz2:      0    7     2.162
from bnz12:     0    7    10.231
from bnz1:      0    8     3.415
from bnz2:      0    8     3.415
from bnz12:     0    8    10.567
from bnz1:      0    9     3.893
from bnz2:      0    9     3.893
from bnz12:     0    9    10.731
from bnz1:      0   10     3.415
from bnz2:      0   10     3.415
from bnz12:     0   10    10.567
from bnz1:

If we print out the xyz files for the three benzene objects, 

In [10]:
$bnz1->print_xyz;
$bnz2->print_xyz;
$bnz3->print_xyz;
1;

12

  C   0.000000   1.402720   0.000000
  C  -1.214790   0.701360   0.000000
  C  -1.214790  -0.701360   0.000000
  C   0.000000  -1.402720   0.000000
  C   1.214790  -0.701360   0.000000
  C   1.214790   0.701360   0.000000
  H   0.000000   2.490290   0.000000
  H  -2.156660   1.245150   0.000000
  H  -2.156660  -1.245150   0.000000
  H   0.000000  -2.490290   0.000000
  H   2.156660  -1.245150   0.000000
  H   2.156660   1.245150   0.000000
12

  C   0.000000   1.402720   0.000000
  C  -1.214790   0.701360   0.000000
  C  -1.214790  -0.701360   0.000000
  C   0.000000  -1.402720   0.000000
  C   1.214790  -0.701360   0.000000
  C   1.214790   0.701360   0.000000
  H   0.000000   2.490290   0.000000
  H  -2.156660   1.245150   0.000000
  H  -2.156660  -1.245150   0.000000
  H   0.000000  -2.490290   0.000000
  H   2.156660  -1.245150   0.000000
  H   2.156660   1.245150   0.000000
12

  C   0.000000   1.402720   0.000000
  C  -1.214790   0.701360   0.000000
  C  -1.214790  -0.701360 

1


we see that the coordinates within the HackaMol::Molecule have not changed; thus, the vectors above are copied in the map into new, distinct memory locations.  In the following exercises, we will play around with the information and coordinates within the HackaMol objects.

### Exercise 1. Play around with the methods listed below to interact with your benzene molecules.
The HackaMol molecule has some methods that are useful for interacting with the atomic data.

Here is a list of methods to try here:
    1.  print_xyz
    2.  print_pdb
    3.  COM
    4.  total_mass
    5.  all_atoms
    6.  get_atoms(0) to get the first atom
    7.  translate
    8.  rotate
    9.  bin_atoms_name
    10. dump
 
Each can be invoked as follows:

$bnz1->method (args)   # with arguments, see documentation of what arguments are.
 
-or-

$bnz2->method (args)

etc.

#### Example 1.0. bin_atoms_name, use the method change a carbon to a Hg and then back again.

In [11]:
say $bnz1->bin_atoms_name; # generate a string from the atom symbols
say $bnz1->total_mass;

$bnz1->get_atoms(1)->change_Z(80); # change an atom to Hg
say $bnz1->bin_atoms_name;
say $bnz1->total_mass;

$bnz1->get_atoms(1)->change_Z(6); # change it back.
say $bnz1->bin_atoms_name;
say $bnz1->total_mass;

C6H6
78.11184
HgC5H6
266.69314
C6H6
78.11184


1


Warning: cleaning atom attributes for in place change. setting atom->is_dirty at reply input line 4.

cleaning atom attributes for in place change. setting atom->is_dirty at reply input line 8.


### Exercise 2. Use the translate method to translate your molecule along the z-axis.  Use the print_xyz method to see the result and then translate it back so that the center of mass is at (0,0,0);

\$mol->translate(-1 * $mol->COM);


### Exercise 3. Create a line of 21 mercury atoms perpendicular to the plane of the benzene molecule, with arbitrary orientation, spanning 10 angstroms above and below the plane of the molecule.  Visualize in VMD.
The framework, which includes solving for the perpendicular vector (using a cross product of two atom vectors) of the problem is below.  You need to adjust it to get the 21 Mercury atoms.  Adjust the rotation and translations and see that it still works!


In [12]:
$bnz1 = $bldr->read_string_mol($c6h6_xyz, 'xyz');  # reset the molecule!
# orient it however you like!
$bnz1->translate( V(10,0,10) );
$bnz1->rotate(    V(1,0,0), 
                     90, 
                $bnz1->COM
); # 90 degree rotation about the x axis

# let's put a Hg at the COM
my $hg0 = HackaMol::Atom->new(symbol => 'Hg', coords => [ V(@{$bnz1->COM}) ] );
$bnz1->push_atoms($hg0); 

# save this COM... when we add Hg to the molecule the COM changes!
my $vec1 = $bnz1->get_atoms(0)->xyz - $hg0->xyz ;  # vector to COM
my $vec2 = $bnz1->get_atoms(2)->xyz - $hg0->xyz ;  # vector to COM

my $perp_vec = ($vec1 x $vec2) -> versor;

foreach my $scale (-1,1){
    my $xyz = $scale*( 2 * $perp_vec) + $hg0->xyz;
    my $hg = HackaMol::Atom->new(symbol => 'Hg', coords => [ $xyz ] );
    $bnz1->push_atoms($hg);
}

$bnz1->print_xyz; 1;

15

  C  10.000000  -0.000000  11.402720
  C   8.785210  -0.000000  10.701360
  C   8.785210  -0.000000   9.298640
  C  10.000000  -0.000000   8.597280
  C  11.214790  -0.000000   9.298640
  C  11.214790  -0.000000  10.701360
  H  10.000000  -0.000000  12.490290
  H   7.843340  -0.000000  11.245150
  H   7.843340  -0.000000   8.754850
  H  10.000000  -0.000000   7.509710
  H  12.156660  -0.000000   8.754850
  H  12.156660  -0.000000  11.245150
 Hg  10.000000  -0.000000  10.000000
 Hg  10.000000   2.000000  10.000000
 Hg  10.000000  -2.000000  10.000000


1


#### Exercise 4. create a pdb with 512 benzene molecules with different residues and resnames, and visualize in VMD.
The framework should get you a pdb (fat_stacks.pdb) that is full of 512 benzenes.  You need to adjust the atom attributes to get each benzene to have different residue number and resnames, which will make your VMDing a little more interesting if you color by residue or name. Your goal is to work with the code below to reproduce the image at the header of this notebook.

You'll need to put this code in the right place (in the code cell below) to get this to work:

```
            foreach my $atom ($bnz->all_atoms){
                # reset atom name  to be the symbol and array location (iatom)  
                my $name = $atom->symbol . $atom->iatom;
                $atom->name($name);
            
                # reset resname
                $atom->resname("BNZ");
            
                # reset resid
                $atom->resid($count);
            }
```

In [13]:
my $count = 0;
my $a = 10;
my $b = 10;
my $c = 5;

my $fat_stacks = HackaMol::Molecule -> new ;

foreach my $ibenz (0 .. 7){

    my $dx = $ibenz * $a;
    foreach my $jbenz (0 .. 7) {
    
        my $dy = $jbenz * $b;
        foreach my $kbenz (0 .. 7){

            my $dz = $kbenz * $c;
            
            my $bnz = $bldr->read_string_mol($c6h6_xyz, 'xyz');
            $bnz->translate(V($dx,$dy,$dz));

            $fat_stacks->push_atoms( $bnz->all_atoms);
            $count++;
            
        }
    }
}
$fat_stacks->print_pdb('fat_stacks.pdb'); 1

1


#### Optional Exercise 5. Use the mercury axis code to make something interesting to look at


In [14]:
my $bnz1 = $bldr->read_string_mol($c6h6_xyz, 'xyz');  # reset the molecule!
# orient it however you like!
$bnz1->translate( V(10,0,10) );
$bnz1->rotate(    V(1,1,0), 
                     90, 
                $bnz1->COM
); # 90 degree rotation about the x axis

# let's put a Hg at the COM
my $hg0 = HackaMol::Atom->new(symbol => 'Hg', coords => [ V(@{$bnz1->COM}) ] );
$bnz1->push_atoms($hg0); 

# save this COM... when we add Hg to the molecule the COM changes!
my $vec1 = $bnz1->get_atoms(0)->xyz - $hg0->xyz ;  # vector to COM
my $vec2 = $bnz1->get_atoms(2)->xyz - $hg0->xyz ;  # vector to COM

my $perp_vec = ($vec1 x $vec2) -> versor;
my $nmax = 20;

foreach my $scale (1 .. $nmax){
    my $xyz = $scale*( 1 * $perp_vec) + $hg0->xyz;
    my $hg = HackaMol::Atom->new(symbol => 'Hg', coords => [ $xyz ] );
    $bnz1->push_atoms($hg);
}

$bnz1->translate(-$bnz1->get_atoms(12+$nmax)->xyz); # 12 includes the first Hg

my $fh = $bnz1->print_xyz('cone.xyz');
foreach (1 .. 359){
    $bnz1->rotate(
        V(0,1,0),
        1,
        V(0,0,0),
    );
    $bnz1->print_xyz($fh);
}
$fh->close;

1


Warning: overwrite cone.xyz at reply input line 28.


In [15]:
my $CAII = HackaMol->new->pdbid_mol('2CBA');
$CAII->translate(-$CAII->COM);
$CAII->translate(V(0,30,0));
$CAII->print_pdb("icecream.pdb");1

1


#### And here are the mercury/benzene cone with the Carbonic Anhydrase icecream on top.
<img src="figs/IceCreamCone.png">